# 동화 텍스트 생성 AI

## 검증 -> 통합 프롬프트 재구성 -> LLM 생성 로직 구현

1단계: 라이브러리 설치 및 설정

In [13]:
# 1. 라이브러리 설치
!pip install openai -q

import os
from openai import OpenAI
import json
from getpass import getpass

# 2. API Key 입력 (실행 시 입력창에 sk-로 시작하는 키를 넣고 엔터를 치세요)
os.environ["OPENAI_API_KEY"] = getpass("OpenAI API Key를 입력하세요: ")

client = OpenAI()

print("✅ 설정이 완료되었습니다.")

OpenAI API Key를 입력하세요: ··········
✅ 설정이 완료되었습니다.


2단계: 동화 생성 엔진 (아키텍처 반영 로직)

In [14]:
def generate_fairy_tale(user_input):
    print(f"\n🚀 [프로세스 시작] 사용자 입력: \"{user_input}\"")

    # ------------------------------------------------------------------
    # Step 1: 입력값 검증 및 분석 (Safety & Requirement Check)
    # ------------------------------------------------------------------
    print("1️⃣ [검증 단계] 입력값의 안전성과 적합성을 판단 중...")

    check_prompt = f"""
    당신은 동화 서비스의 '안전 관리자'입니다.
    사용자의 입력이 동화를 만들기에 적절한지 판단하세요.
    폭력적, 선정적, 혐오 표현이 있거나 동화와 전혀 관련 없는 요청이라면 'FALSE'를,
    적절하다면 'TRUE'를 출력하세요.
    설명 없이 오직 TRUE 또는 FALSE만 대답하세요.

    사용자 입력: {user_input}
    """

    check_response = client.chat.completions.create(
        model="gpt-4o",
        messages=[{"role": "system", "content": "You are a safety moderator."},
                  {"role": "user", "content": check_prompt}],
        temperature=0.0
    )

    is_safe = check_response.choices[0].message.content.strip()

    if "FALSE" in is_safe:
        # Changed to return two None values to avoid TypeError in caller
        print("⚠️ [차단됨] 부적절하거나 동화 생성과 관련 없는 요청입니다. 다시 입력해주세요.")
        return None, None

    print("   -> ✅ 검증 통과.")


    # ------------------------------------------------------------------
    # Step 2: 통합 프롬프트 재구성 (Prompt Re-engineering)
    # 사용자의 짧은 입력을 상세한 '동화 기획서'로 변환합니다.
    # ------------------------------------------------------------------
    print("2️⃣ [기획 단계] 사용자 의도를 분석하여 '통합 프롬프트' 생성 중...")

    architect_prompt = f"""
    당신은 '한국형 동화 기획자'입니다.
    사용자 입력을 분석하여 시대 배경을 정하고, 그 시대에 맞지 않는 설정은 **반드시** 해당 시대에 어울리는 개념으로 바꿔야 합니다.

    [필수 수행 절차]
    1. **시대 결정(Era)**: [단군시대, 조선시대, 근대, 현대] 중 택 1
       - 입력에 시대 언급이 없으면, 내용에 가장 잘 어울리는 시대를 고르되 기본은 '현대'입니다.

    2. **시대별 개념 치환 (Context Mapping)**:
       **만약 '현대(Modern)'가 선택되었다면:**
       - 왕, 왕비 -> '시장님', '교장 선생님', '회장님', '부모님'
       - 공주, 왕자 -> '인기 아이돌', '부잣집 친구', '반장', '전학생'
       - 무도회 -> '학예회', '학교 축제', '생일 파티', '장기자랑'
       - 왕국, 성 -> '학교', '우리 동네', '아파트', '놀이공원'
       - 마법 -> '과학 기술', '신기한 도구', '스마트폰', '꿈속 상상' (혹은 현대 판타지로 처리)

       **만약 '조선시대'가 선택되었다면:**
       - 파티 -> '잔치', '연회'
       - 빵/케이크 -> '떡', '다과'
       - 드레스 -> '비단 한복', '당의'

    [사용자 입력]
    {user_input}

    [출력 포맷 - JSON 형식만 출력할 것]
    {{
      "title": "동화 제목",
      "era": "선택한 시대 (단군시대/조선시대/근대/현대)",
      "characters": "주인공과 등장인물 상세 설정 (한국식 이름, 직업, 성격 서술)",
      "setting": "배경 설정 (구체적인 한국 지명 포함)",
      "korean_items": ["이야기에 사용할 한국적 소재1", "소재2", "소재3"],
      "magic_trigger": "변신이나 사건이 발생하는 구체적인 계기 (예: 경복궁에서 주운 신비한 노리개, 남산타워의 소원 자물쇠가 빛나며 등)",
      "tone": "분위기",
      "moral": "교훈",
      "plot_outline": "줄거리 요약"
    }}
    """

    plan_response = client.chat.completions.create(
        model="gpt-4o",
        messages=[{"role": "system", "content": "You are a creative story architect. Output JSON only."},
                  {"role": "user", "content": architect_prompt}],
        response_format={ "type": "json_object" },
        temperature=0.7
    )

    story_plan = json.loads(plan_response.choices[0].message.content)

    # Call generate_story_text and return its results
    final_story, final_plan = generate_story_text(story_plan, client)
    return final_story, final_plan

def generate_story_text(story_plan, client):
    # ------------------------------------------------------------------
    # [중간 로직] 시대별 오프닝 문구 매핑 (수정된 버전)
    # ------------------------------------------------------------------
    # 1. 시대별 오프닝 문구 정의 (Tone & Manner: 따뜻함, 환상적, 운율감)
    OPENING_PHRASES = {
        "myth": "하늘이 처음 열리고 곰과 호랑이가 소원을 빌던 아주 먼 옛날,",             # 단군시대
        "traditional": "호랑이가 곰방대를 물고 뻐끔뻐끔 연기를 내뿜던 시절,",              # 조선시대
        "modern_history": "전차 종소리가 땡땡 울리고 신기한 물건이 가득하던 시절,",       # 근대
        "contemporary": "높은 빌딩이 숲을 이루고 우리가 살고 있는 바로 지금,"
    }

    # 2. 기획된 era에 맞는 오프닝 가져오기 (매칭 안 될 경우 'traditional' 기본값)
    # story_plan['era']에는 'myth', 'traditional' 등의 키값이 들어있다고 가정
    selected_era_key = None
    era_from_plan = story_plan.get("era", "")
    if "단군" in era_from_plan:
        selected_era_key = "myth"
    elif "조선" in era_from_plan:
        selected_era_key = "traditional"
    elif "근대" in era_from_plan:
        selected_era_key = "modern_history"
    elif "현대" in era_from_plan:
        selected_era_key = "contemporary"
    else:
        selected_era_key = "traditional" # Default if no clear era match

    opening_sentence = OPENING_PHRASES.get(selected_era_key, OPENING_PHRASES["traditional"])

    print("\n   📄 [생성된 동화 설계도]")
    print(f"   - 제목: {story_plan.get('title', '제목 미정')}")
    print(f"   - 시대: {selected_era_key} -> 첫 문장: \"{opening_sentence}\"")
    print(f"   - 한국적 소재: {story_plan.get('korean_items')}")

    # ------------------------------------------------------------------
    # Step 3: 동화 텍스트 생성 (LLM Generation)
    # ------------------------------------------------------------------
    print("\n3️⃣ [생성 단계] 설계도를 바탕으로 GPT-4o가 동화를 집필 중입니다...")

    # 프롬프트 강화: 첫 문장 강제 지침을 최상단에 배치하여 중요도 강조
    writer_prompt = f"""
    당신은 한국의 정서를 담아 따뜻하고 교훈적인 이야기를 만드는 베스트셀러 동화 작가입니다.
    아래의 [동화 설계도]를 바탕으로 아이들이 읽기 좋은 동화를 작성해 주세요.

    [작성 가이드]
    1. 필수 지침(매우 중요): 이야기는 무조건 다음 문장으로 시작해야 합니다. 다른 서두를 붙이지 마세요.
       -> "{opening_sentence}"

    2. 한국적 요소 활용: 기획된 한국적 소재({story_plan.get('korean_items')})를 이야기에 자연스럽게 녹여내세요.
       (예: 파티 대신 잔치, 빵 대신 떡, 왕자님 대신 도령님 사용)

    3. 문체 및 어조:
       - '~했어요', '~였답니다'와 같은 부드러운 구어체(경어)를 사용하세요.
       - 아이들에게 읽어주는 듯한 다정한 어조를 유지하세요.

    5. 캐릭터 소개 및 문장 구성 주의사항:

    6. 논리적 연결 (인과관계 강화):
       - 문장과 문장 사이가 뚝 끊기지 않게 연결하세요.
       - A사건이 있었기에 B사건이 일어났음을 독자가 알 수 있어야 합니다.
       - 추천 접속 표현: '그러자', '그 때문에', '덕분에', '마침내', '놀랍게도'

    7. 구성과 분량:
       - 기승전결(도입-전개-위기-절정-결말)이 뚜렷해야 합니다.
       - 분량: 낭독 시 약 3~5분 분량 (공백 포함 약 1,200자 ~ 1,500자).

    [동화 설계도]
    {json.dumps(story_plan, ensure_ascii=False)}
    """

    story_response = client.chat.completions.create(
        model="gpt-4o",
        messages=[
            {"role": "system", "content": "You are a warm and creative fairy tale writer specialized in Korean folklore."},
            {"role": "user", "content": writer_prompt}
        ],
        temperature=0.7  # 창의성을 유지하되 너무 튀지 않도록 0.7~0.8 추천
    )

    final_story = story_response.choices[0].message.content

    return final_story, story_plan

# 함수 테스트
print("✅ 동화 생성 엔진 준비 완료 (한국적 요소 및 시대별 도입부 적용)")

✅ 동화 생성 엔진 준비 완료 (한국적 요소 및 시대별 도입부 적용)


## 생성 → 후처리 검증(유해성/저작권/품질) → (미통과 시) 피드백 루프 → 최종 포맷팅 구현

3단계: 후처리 검증 및 재생성

In [15]:
import json
import re
import os
import pytz
from datetime import datetime
from google.colab import files
from openai import OpenAI

# client = OpenAI(...) # 클라이언트 초기화 부분은 기존 코드 유지

# ==============================================================================
# Step 4: 후처리 검증 (Editor AI)
# ==============================================================================
def verify_story_quality(story_text, story_plan):
    print("\n🧐 [검증 단계] '수석 편집장 AI'가 루브릭에 맞춰 원고를 정밀 심사 중입니다...")

    # 사용자가 입력한 교훈 (없을 경우 대비)
    target_moral = story_plan.get('moral', '교훈 없음')

    # 상세 루브릭이 포함된 시스템 프롬프트
    verification_prompt = f"""
    당신은 5~8세 아동용 동화 서비스 'Once Upon a Time'의 [수석 품질 검수자]입니다.
    작가(AI)가 작성한 동화 원고를 아래 [검증 루브릭]에 따라 엄격하게 1~5점 척도로 채점하고, 최종 판정을 내리세요.

    [검증 루브릭 (Scoring Rubric)]


    1. 유해성 (Safety) - Critical
    - 기준: 아동에게 정서적 충격, 공포, 혐오감을 주는가?
    - 5점: 부정적 요소 전무, 심리적 안정감을 주는 따뜻한 내용.
    - 4점: 갈등이 있으나 권선징악으로 건전하게 해소됨.
    - 3점: 약간의 거친 표현(예: "멍청이", 가벼운 다툼)이 있으나 맥락상 허용 가능.
    - 2점: [Fail] 따라 할 수 있는 나쁜 행동, 구체적인 공포 묘사 포함.
    - 1점: [Fail] 잔혹성, 선정성, 혐오 표현, 트라우마 유발.

    2. 저작권 (Copyright) - Critical
    - 기준: 기존 유명 IP(디즈니, 해리포터 등)의 고유명사나 설정을 도용했는가?
    - 5점: 고유명사, 플롯, 캐릭터가 완전히 오리지널함.
    - 4점: 전형적 클리셰(왕자, 공주)는 있으나 특정 작품이 연상되지 않음.
    - 3점: 구조가 유명 동화와 흡사하지만 고유명사는 다름.
    - 2점: [Fail] 특정 IP(예: 엘사, 호그와트, 피카츄)를 연상시키는 명칭이나 설정 등장.
    - 1점: [Fail] 기존 작품의 이름, 대사, 스토리를 그대로 베낌.

    3.  품질: 교훈 적합성 (Alignment)
    - 기준: 사용자(부모)가 입력한 [요청 교훈]이 이야기에 잘 녹아있는가?
    - 5점: 교훈이 결말과 완벽히 연결되며 아이가 스스로 깨달을 수 있음.
    - 4점: 교훈이 잘 드러나나, 다소 직접적인 설명 방식임.
    - 3점: 교훈이 포함되어 있으나 이야기 흐름과 약간 겉돔.
    - 2점: [Warning] 교훈이 억지스럽거나 미미함.
    - 1점: [Warning] 요청된 교훈과 무관한 내용.

    4.  품질: 정서 및 표현 (Aesthetics)
    - 기준: 한국적 정서(존댓말)와 아동 눈높이 표현(의성어/의태어)이 적절한가?
    - 5점: 부드러운 '해요체', 풍부한 의성어/의태어, 따뜻한 어조.
    - 4점: 문장이 매끄럽고 이해하기 쉬운 단어 사용.
    - 3점: 문법은 맞으나 딱딱하거나 교과서적인 문체.
    - 2점: [Warning] 번역투 문장, 어려운 한자어 사용.
    - 1점: [Warning] 비문이 많거나 문맥이 끊김.

    [원고 내용]
    {story_text}

    [출력 포맷 - 반드시 JSON 형식만 출력]
    {{
        "scores": {{
            "safety": Integer (1-5),
            "copyright": Integer (1-5),
            "alignment": Integer (1-5),
            "aesthetics": Integer (1-5)
        }},
        "reasons": {{
            "safety": "점수 부여 사유 (한 문장)",
            "copyright": "점수 부여 사유",
            "alignment": "점수 부여 사유",
            "aesthetics": "점수 부여 사유"
        }},
        "final_decision": "PASS" 또는 "FAIL",
        "rewrite_suggestion": "FAIL인 경우, 작가에게 전달할 구체적인 수정 지시사항 (PASS면 null)"
    }}
    ** 주의: 유해성이나 저작권 점수가 3점 미만이면 무조건 'final_decision'은 "FAIL"입니다. **
    """

    try:
        response = client.chat.completions.create(
            model="gpt-4o",
            messages=[
                {"role": "system", "content": "You are a strict editor evaluator. Output JSON only."},
                {"role": "user", "content": verification_prompt}
            ],
            response_format={ "type": "json_object" },
            temperature=0.0 # 평가의 일관성을 위해 0으로 설정
        )

        result = json.loads(response.choices[0].message.content)

        # 로그 출력 (디버깅용)
        print(f"   -> 판정 결과: {result['final_decision']}")
        print(f"   -> 점수: Safety({result['scores']['safety']}), Copyright({result['scores']['copyright']}), Quality({result['scores']['alignment']}/{result['scores']['aesthetics']})")

        return result

    except Exception as e:
        print(f"검증 중 오류 발생: {e}")
        # 오류 발생 시 안전을 위해 FAIL 처리하거나 예외 처리
        return {"final_decision": "FAIL", "rewrite_suggestion": "시스템 오류로 검증 실패"}

# ==============================================================================
# Step 5: 피드백 반영 재작성 (Refinement)
# ==============================================================================
def refine_story(previous_story, feedback, story_plan):
    print(f"\n✍️ [수정 단계] 작가 AI가 피드백을 반영하여 원고를 다시 쓰고 있습니다...")

    refine_prompt = f"""
    당신은 동화 작가입니다. 초안이 편집 심사에서 반려되었습니다.
    아래 피드백을 반영하여 동화를 다시 작성해주세요.

    [기획 정보]
    제목: {story_plan['title']}
    교훈: {story_plan['moral']}

    [편집장 피드백 (수정 요청)]
    {feedback}

    [기존 원고]
    {previous_story}

    위 내용을 바탕으로 피드백을 해결한 완성된 동화를 다시 써주세요.
    (문체와 분량은 기존 가이드를 따르세요.)
    """

    response = client.chat.completions.create(
        model="gpt-4o",
        messages=[{"role": "system", "content": "You are a professional fairy tale writer."},
                  {"role": "user", "content": refine_prompt}],
        temperature=0.7
    )

    return response.choices[0].message.content

# ==============================================================================
# [Helper] 스토리 텍스트 파싱 (Text -> Script Structure)
# ==============================================================================
def parse_text_to_script(full_text):
    """
    LLM이 생성한 줄글 텍스트를 분석하여 (화자, 대사, 감정) 구조로 변환합니다.
    GPT-SoVITS 추론을 위해 문장 단위로 쪼개는 것이 유리합니다.
    """
    script_data = []
    lines = full_text.split('\n')
    seq_counter = 1

    for line in lines:
        line = line.strip()
        if not line: continue  # 빈 줄 건너뛰기

        # 로직: "화자: 대사 (감정)" 또는 "화자: 대사" 형식을 파싱
        # 예: "호랑이: 어흥! 배가 고프다. (화남)" -> Role: 호랑이, Text: 어흥!..., Emotion: 화남

        # 1. 화자와 대사 분리 (콜론 : 기준)
        if ":" in line:
            parts = line.split(":", 1)
            role = parts[0].strip()
            text_content = parts[1].strip()
        else:
            role = "narrator" # 콜론이 없으면 해설(Narrator)로 간주
            text_content = line

        # 2. 감정 추출 (괄호 안의 내용 추출 시도)
        emotion = "neutral" # 기본값
        emotion_match = re.search(r'\((.*?)\)', text_content)
        if emotion_match:
            emotion_candidate = emotion_match.group(1)
            # 감정 키워드가 맞는지 확인하는 로직이 들어갈 수 있음 (AI 허브 라벨링 매칭)
            emotion = emotion_candidate
            # 텍스트에서 감정 지문 제거 (TTS가 읽지 않도록)
            text_content = re.sub(r'\(.*?>\)', '', text_content).strip()

        script_item = {
            "seq": seq_counter,
            "role": role,  # 프론트엔드에서 캐릭터 선택 시 사용
            "text": text_content, # TTS 입력 텍스트
            "emotion": emotion, # AI 허브 데이터 매핑용 (슬픔, 기쁨, 화남 등)
            "audio_file_name": f"{seq_counter:03d}_{role}.wav" # 추후 생성될 파일명 미리 정의
        }

        script_data.append(script_item)
        seq_counter += 1

    return script_data

# ==============================================================================
# [수정됨] 최종 결과 포맷팅 및 JSON 구조화
# ==============================================================================
# 기존 def format_final_output(story_text, story_plan): 을 아래처럼 변경하세요.

def format_final_output(story_text, story_plan, original_input):
    """
    인자 설명:
    - story_text: LLM이 만든 동화 텍스트
    - story_plan: LLM이 만든 기획안
    - original_input: ★ 프론트엔드에서 받은 원본 JSON (여기에 모험, 제목 등이 들어있음)
    """

    # 1. 텍스트 파싱
    structured_script = parse_text_to_script(story_text)

    # 2. 한국 시간
    kst = pytz.timezone('Asia/Seoul')
    now = datetime.now(kst)

    # 3. 태그 정리 (Vibe 포함)
    general_tags = [original_input.get('vibe', '따뜻함'), "교훈", "어린이"]

    # 4. 결과 JSON 조립
    final_json = {
        "metadata": {
            "project_name": "Once Upon a Time (P-실무)",
            "version": "v1.0_GPT-SoVITS",
            "created_at": now.strftime("%Y-%m-%d %H:%M:%S"),
            "model_type": "gpt_sovits_v2",
            "total_segments": len(structured_script)
        },
        "story_info": {
            # ★ 여기가 핵심! LLM이 만든 것보다 입력값(original_input)을 우선 사용
            "title": original_input.get("title", "제목 없음"),
            "theme": original_input.get("theme", "자유 주제"), # 입력한 '모험'이 들어감
            "vibe": original_input.get("vibe", "일반"),      # 입력한 '따뜻함'이 들어감
            "original_prompt": original_input.get("prompt", ""),

            "target_age": story_plan.get("target_age", "All"),
            "summary": story_plan.get("plot_outline", ""),
            "keywords": story_plan.get("korean_items", [])[:3]
        },
        "script": structured_script,

        # (기타 필드 유지)
        "content": story_text,
        "verification_status": "Verified"
    }

    return final_json

# ==============================================================================
# [수정됨] JSON 파일 저장 함수 (Colab 호환성 강화)
# ==============================================================================
def save_story_to_json(json_data):
    try:
        # Debug print to inspect json_data before access
        print(f"Debug: json_data keys in save_story_to_json: {json_data.keys()}")
        if "story_info" not in json_data:
            print("Debug: 'story_info' key is MISSING in json_data!")

        # Safely access title using .get() to prevent KeyError
        title = json_data.get("story_info", {}).get("title", "제목 없음")
        # If title is still '제목 없음', try getting from the top level (added for robustness)
        if title == "제목 없음":
            title = json_data.get("title", "제목 없음")

        # 파일명 안전하게 변환 (공백 -> _, 특수문자 제거)
        safe_title = "".join([c if c.isalnum() else "_" for c in title])
        kst = pytz.timezone('Asia/Seoul') # 한국 시간대 설정
        timestamp = datetime.now(kst).strftime("%Y%m%d_%H%M") # 파일명용 타임스탬프
        file_name = f"{safe_title}_{timestamp}.json"

        # 파일 쓰기
        with open(file_name, 'w', encoding='utf-8') as f:
            json.dump(json_data, f, ensure_ascii=False, indent=4)

        print(f"\n💾 [System] JSON 파일이 저장되었습니다: {file_name}")

        # Colab 다운로드 처리
        if os.path.exists(file_name):
            files.download(file_name)
        else:
            print("⚠️ 파일 생성 확인 불가 (새로고침 필요)")

    except Exception as e:
        print(f"❌ 파일 저장 중 오류 발생: {e}")

4단계: 메인 실행기-전체 프로세스 통합 및 출력 제어

In [16]:
import json

# ==============================================================================
# [Helper] 점수표 출력 함수 (시각적 효과)
# ==============================================================================
def print_score_card(attempt, result):
    scores = result.get('scores', {})
    reasons = result.get('reasons', {})
    decision = result.get('final_decision', 'UNKNOWN')

    print(f"\n📊 [검증 리포트 - 시도 {attempt}회차]")
    print("-" * 50)
    print(f"1. 유해성 (Safety)    : {'★' * scores.get('safety', 0)} ({scores.get('safety', 0)}/5)")
    print(f"2. 저작권 (Copyright) : {'★' * scores.get('copyright', 0)} ({scores.get('copyright', 0)}/5)")
    print(f"3. 교훈성 (Alignment) : {'★' * scores.get('alignment', 0)} ({scores.get('alignment', 0)}/5)")
    print(f"4. 심미성 (Aesthetics): {'★' * scores.get('aesthetics', 0)} ({scores.get('aesthetics', 0)}/5)")
    print("-" * 50)
    print(f"🧐 종합 판정: [{'✅ PASS' if decision == 'PASS' else '❌ FAIL'}]")

    if decision == 'FAIL':
        # Display all relevant reasons if the decision is FAIL
        fail_reasons = []
        for key, value in scores.items():
            if value < 3: # Assuming 3 is the minimum passing score for safety/copyright
                if key == 'safety' or key == 'copyright': # Critical failures
                    fail_reasons.append(reasons.get(key, ''))
            elif key == 'alignment' or key == 'aesthetics': # Warnings for quality scores below 3
                if value < 3:
                    fail_reasons.append(reasons.get(key, ''))

        if not fail_reasons: # Fallback if no specific reason found
            fail_reasons.append('기준 미달')

        print(f"⚠️ 사유: {', '.join([r for r in fail_reasons if r])}")
    print("=" * 60)


# ==============================================================================
# [메인 실행기] 전체 프로세스 통합
# ==============================================================================
def process_frontend_request(json_data):
    """
    프론트엔드 JSON을 파싱하여 LLM에 들어갈 최종 프롬프트를 구성하고 실행합니다.
    """
    # 1. JSON 데이터 파싱
    theme = json_data.get("theme", "자유 주제")
    vibe = json_data.get("vibe", "일반적인")
    user_prompt = json_data.get("prompt", "")
    title = json_data.get("title", "제목 없음")

    print(f"📥 [System] 요청 수신: {title} ({theme}, {vibe})")

    # 2. 프롬프트 구성
    enriched_prompt = f"""
    [동화 생성 요청]
    1. 제목: {title}
    2. 핵심 주제: {theme}
    3. 이야기 분위기(Tone & Manner): {vibe}
    4. 사용자 입력 줄거리: {user_prompt}

    위 4가지 요소를 모두 반영하여, 아이들이 읽기 좋은 동화를 작성해줘.
    특히 '{vibe}' 분위기를 살려서 서술해줘.
    """

    max_retries = 3
    attempt = 0
    initial_story_and_plan_set = False
    current_story = None
    story_plan = None
    feedback = None
    is_finalized = False

    # 3. 생성 루프
    while attempt < max_retries:
        attempt += 1

        if not initial_story_and_plan_set:
            print(f"\n📝 [Step 1] 초안 작성 중... (Attempt {attempt})")
            generated_story, generated_plan = generate_fairy_tale(enriched_prompt)

            if generated_story is None or generated_plan is None:
                print("❌ [System] 생성 실패.")
                return None

            current_story = generated_story
            story_plan = generated_plan
            initial_story_and_plan_set = True

            print("\n" + "┌" + "─"*58 + "┐")
            print("│  📜 [초안]                                           │")
            print("└" + "─"*58 + "┘")
            print(current_story[:100] + "...") # 로그 줄임
            print("-" * 60)

        else:
            print(f"\n🔄 [Step 1-Retry] 재작성 중... (Attempt {attempt})")
            current_story = refine_story(current_story, feedback, story_plan)

        # 검증
        print("\n🔍 [Step 2] 품질 검증 수행 중...")
        verification_result = verify_story_quality(current_story, story_plan)
        print_score_card(attempt, verification_result)

        decision = verification_result.get('final_decision', 'FAIL')

        if decision == "PASS":
            print("\n✅ [Step 3] 최종안 확정.")
            is_finalized = True
            break
        else:
            feedback = verification_result.get('rewrite_suggestion', '내용 수정 필요')
            if attempt < max_retries:
                print(f"🚫 [System] 기준 미달 -> 재작성")
            else:
                print("❌ [System] 수정 한도 초과.")

    # 4. 최종 결과 처리
    if is_finalized:
        print("\n" + "="*60)
        print("🎉 [Final Result] 최종 완성")
        print("="*60)

        # ★★★ 핵심 수정 부분 ★★★
        # 인자 3개 전달: (스토리, 기획안, "프론트엔드 입력값")
        final_json_data = format_final_output(current_story, story_plan, json_data)

        # 저장
        save_story_to_json(final_json_data)
        return final_json_data
    else:
        print("\n⚠️ 최종 생성 실패.")
        return None

실행 테스트

In [24]:
if __name__ == "__main__":
    # ---------------------------------------------------------
    # 실제 프론트엔드 연동 전, 테스트를 위한 가상의 JSON 데이터
    # (나중에는 이 부분을 API 서버의 Request Body로 받게 됩니다)
    # ---------------------------------------------------------
    test_json_input = {
        "theme": "모험",
        "vibe": "신비로운",
        "prompt": "동대문애서 공주 옷을 찾는 유민이",
        "title": "동대문에 간 공주"
    }

    # 테스트 실행
    print("--- [테스트] 프론트엔드 요청 시뮬레이션 ---")
    process_frontend_request(test_json_input)

--- [테스트] 프론트엔드 요청 시뮬레이션 ---
📥 [System] 요청 수신: 동대문에 간 공주 (모험, 신비로운)

📝 [Step 1] 초안 작성 중... (Attempt 1)

🚀 [프로세스 시작] 사용자 입력: "
    [동화 생성 요청]
    1. 제목: 동대문에 간 공주
    2. 핵심 주제: 모험
    3. 이야기 분위기(Tone & Manner): 신비로운
    4. 사용자 입력 줄거리: 동대문애서 공주 옷을 찾는 유민이

    위 4가지 요소를 모두 반영하여, 아이들이 읽기 좋은 동화를 작성해줘.
    특히 '신비로운' 분위기를 살려서 서술해줘.
    "
1️⃣ [검증 단계] 입력값의 안전성과 적합성을 판단 중...
   -> ✅ 검증 통과.
2️⃣ [기획 단계] 사용자 의도를 분석하여 '통합 프롬프트' 생성 중...

   📄 [생성된 동화 설계도]
   - 제목: 동대문에 간 유민이
   - 시대: contemporary -> 첫 문장: "높은 빌딩이 숲을 이루고 우리가 살고 있는 바로 지금,"
   - 한국적 소재: ['한복', '노리개', '전통 시장']

3️⃣ [생성 단계] 설계도를 바탕으로 GPT-4o가 동화를 집필 중입니다...

┌──────────────────────────────────────────────────────────┐
│  📜 [초안 (Draft Version)]                           │
└──────────────────────────────────────────────────────────┘
높은 빌딩이 숲을 이루고 우리가 살고 있는 바로 지금, 서울의 동대문 시장은 여전히 사람들로 북적이고 있었어요. 그날은 유난히 맑은 날씨였고, 유민이는 언니 수연과 함께 동대문 시장으로 나들이를 갔답니다. 유민이는 초등학교 5학년인데, 호기심이 많고 모험심이 강한 아이였어요. 그래서 전통 시장의 다양한 물건들과 사람들의 활기찬 모습이 유민이의 마음을 더욱 설레게 했지요.

시장

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

# JSON 파일 저장 및 다운로드 코드

In [18]:
import json
import re
import os
import pytz
from datetime import datetime
from google.colab import files

# ==============================================================================
# [Helper] 스토리 텍스트 파싱 (Text -> Script Structure)
# ==============================================================================
def parse_text_to_script(full_text):
    """
    LLM이 생성한 줄글 텍스트를 분석하여 (화자, 대사, 감정) 구조로 변환합니다.
    GPT-SoVITS 추론을 위해 문장 단위로 쪼개는 것이 유리합니다.
    """
    script_data = []
    lines = full_text.split('\n')
    seq_counter = 1

    for line in lines:
        line = line.strip()
        if not line: continue  # 빈 줄 건너뛰기

        # 로직: "화자: 대사 (감정)" 또는 "화자: 대사" 형식을 파싱
        # 예: "호랑이: 어흥! 배가 고프다. (화남)" -> Role: 호랑이, Text: 어흥!..., Emotion: 화남

        # 1. 화자와 대사 분리 (콜론 : 기준)
        if ":" in line:
            parts = line.split(":", 1)
            role = parts[0].strip()
            text_content = parts[1].strip()
        else:
            role = "narrator" # 콜론이 없으면 해설(Narrator)로 간주
            text_content = line

        # 2. 감정 추출 (괄호 안의 내용 추출 시도)
        emotion = "neutral" # 기본값
        emotion_match = re.search(r'\((.*?)\)', text_content)
        if emotion_match:
            emotion_candidate = emotion_match.group(1)
            # 감정 키워드가 맞는지 확인하는 로직이 들어갈 수 있음 (AI 허브 라벨링 매칭)
            emotion = emotion_candidate
            # 텍스트에서 감정 지문 제거 (TTS가 읽지 않도록)
            text_content = re.sub(r'\(.*?>\)', '', text_content).strip()

        # 3. 스크립트 객체 생성
        script_item = {
            "seq": seq_counter,
            "role": role,  # 프론트엔드에서 캐릭터 선택 시 사용
            "text": text_content, # TTS 입력 텍스트
            "emotion": emotion, # AI 허브 데이터 매핑용 (슬픔, 기쁨, 화남 등)
            "audio_file_name": f"{seq_counter:03d}_{role}.wav" # 추후 생성될 파일명 미리 정의
        }

        script_data.append(script_item)
        seq_counter += 1

    return script_data

# ==============================================================================
# [수정됨] 최종 결과 포맷팅 (입력값 + AI생성값 병합)
# ==============================================================================
def format_final_output(story_text, story_plan, original_input):
    """
    story_text: LLM이 쓴 동화 본문
    story_plan: LLM이 짠 기획안 (줄거리, 키워드 등)
    original_input: ★ 프론트엔드 입력값 (여기에 사용자가 입력한 제목, 테마가 있음)
    """

    # 1. 텍스트 파싱
    structured_script = parse_text_to_script(story_text)

    # 2. 시간 설정
    kst = pytz.timezone('Asia/Seoul')
    now = datetime.now(kst)

    # 3. 키워드 추출
    korean_items = story_plan.get('korean_items', [])
    keywords_list = korean_items[:3] if isinstance(korean_items, list) else []

    # 4. 태그 구성 (Vibe 포함)
    general_tags = [original_input.get('vibe', '일반'), "교훈", "어린이"]
    for k in keywords_list:
        if k not in general_tags:
            general_tags.append(k)

    # 5. 최종 JSON 조립
    final_json = {
        "metadata": {
            "project_name": "Once Upon a Time (P-실무)",
            "version": "v1.0_GPT-SoVITS",
            "created_at": now.strftime("%Y-%m-%d %H:%M:%S"),
            "model_type": "gpt_sovits_v2",
            "total_segments": len(structured_script)
        },
        "story_info": {
            # [A] 프론트엔드 입력값 우선 적용 (입력한 '모험', '따뜻함' 유지)
            "title": original_input.get("title", story_plan.get("title", "제목 없음")),
            "theme": original_input.get("theme", "자유 주제"),  # <-- 입력값 '모험'
            "vibe": original_input.get("vibe", "일반"),       # <-- 입력값 '따뜻함'
            "original_prompt": original_input.get("prompt", ""),

            # [B] AI 생성값 적용 (줄거리, 연령, 키워드)
            "target_age": story_plan.get("target_age", "All"),
            "summary": story_plan.get("plot_outline", ""),
            "keywords": keywords_list
        },
        "script": structured_script,

        # [하위 호환성 필드]
        "content": story_text,
        "verification_status": "Verified",
        "tags": general_tags,
        "category": "창작동화",
        "moral": story_plan.get("moral", ""),
        "characters": story_plan.get("characters", {})
    }

    return final_json

# ==============================================================================
# [수정됨] JSON 파일 저장 함수 (한국 시간 파일명 적용)
# ==============================================================================
def save_story_to_json(json_data):
    try:
        # Debug print to inspect json_data before access
        print(f"Debug: json_data keys in save_story_to_json: {json_data.keys()}")
        if "story_info" not in json_data:
            print("Debug: 'story_info' key is MISSING in json_data!")

        # Safely access title using .get() to prevent KeyError
        title = json_data.get("story_info", {}).get("title", "제목 없음")

        # 파일명 안전하게 변환 (공백 -> _, 특수문자 제거)
        safe_title = "".join([c if c.isalnum() else "_" for c in title])

        # 한국 시간대 설정 및 타임스탬프 생성
        kst = pytz.timezone('Asia/Seoul') # 한국 시간대 설정
        timestamp = datetime.now(kst).strftime("%Y%m%d_%H%M") # 파일명용 타임스탬프

        file_name = f"{safe_title}_{timestamp}.json"

        # 파일 쓰기
        with open(file_name, 'w', encoding='utf-8') as f:
            json.dump(json_data, f, ensure_ascii=False, indent=4)

        print(f"\n💾 [System] JSON 파일이 저장되었습니다: {file_name}")

        # Colab 다운로드 처리
        if os.path.exists(file_name):
            files.download(file_name)
        else:
            print("⚠️ 파일 생성 확인 불가 (새로고침 필요)")

    except Exception as e:
        print(f"❌ 파일 저장 중 오류 발생: {e}")